In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class BernoulliBandit:
    def __init__(self, K):
        self.probs = np.random.uniform(size = K)
        self.best_idx = np.argmax(self.probs)
        self.best_prob = self.probs[self.best_idx]
        self.K = K
        
    def step(self, k):
        if np.random.rand() < self.probs[k] :
            return 1
        else :
            return 0

In [ ]:
class Solver:
    def __init__(self, bandit):
        self.bandit = bandit
        self.counts = np.zeros(self.bandit.K)
        self.regret = 0.
        self.actions = []
        self.regrets = []
        
    def update_regret(self, k):
        self.regret += self.bandit.best_prob - self.bandit.probs[k]
        self.regrets.append(self.regret)
        
    def run_one_step(self):
        raise NotImplementedError
    
    def run(self, num_steps):
        for _ in range(num_steps):
            k = self.run_one_step()
            self.counts[k] += 1
            self.actions.append(k)
            self.update_regret(k)

In [ ]:
class EpsilonGreedy(Solver):
    def __init__(self, bandit, epsilon=0.01, init_prob= 0.):
        super(EpsilonGreedy, self).__init__(bandit)
        self.epsilon = epsilon
        self.estimates = np.array([init_prob] * self.bandit.K)
        
    def run_one_step(self):
        if np.random.random() < self.epsilon:
            k = np.random.randint(0, self.bandit.K)
        else :
            k = np.argmax(self.estimates)
        r = self.bandit.step(k)
        self.estimates[k] += 1. /(self.counts[k] + 1) * (r - self.estimates[k])
        return k

In [ ]:
def plot_results(solvers, solver_names):
    for idx, solver in enumerate(solvers):
        time_list = range(len(solver.regrets))
        plt.plot(time_list, solver.regrets, label=solver_names[idx])
    plt.xlabel('Time steps')
    plt.ylabel('Cumulative regrets')
    plt.title('%d-armed bandit' % solvers[0].bandit.K)
    plt.legend()
    plt.show()

In [ ]:
class DecayingEpsilonGreedy(Solver):
    def __init__(self, bandit, epsilon=0.01, init_prob=0.):
        super(DecayingEpsilonGreedy, self).__init__(bandit)
        self.epsilon = epsilon
        self.estimates = np.array([init_prob] * self.bandit.K)
        self.total_count = 0
    
    def run_one_step(self):
        self.total_count += 1
        if np.random.random() < self.epsilon*(0.999)**(self.total_count):
            k = np.random.randint(0, self.bandit.K)
        else :
            k = np.argmax(self.estimates)
        r = self.bandit.step(k)
        self.estimates[k] += 1. /(self.counts[k] + 1) * (r - self.estimates[k])
        return k

In [ ]:
np.random.seed(1)
K = 10
bandit = BernoulliBandit(K)
decaying_epsilon_greedy_solver = DecayingEpsilonGreedy(bandit, 5)
decaying_epsilon_greedy_solver.run(10000)
print('epsilon-贪婪算法的累计懊悔为：', decaying_epsilon_greedy_solver.regret)
plot_results([decaying_epsilon_greedy_solver], ['DecayingEpsilonGreedy'])

In [ ]:
class UCB(Solver):
    def __init__(self, bandit, coef, init_prob=0.):
        super(UCB, self).__init__(bandit)
        self.total_count = 0
        self.estimates = np.array([init_prob] * self.bandit.K)
        self.coef = coef
        
    def run_one_step(self):
        self.total_count += 1
        ucb = self.estimates + self.coef * np.sqrt(np.log(self.total_count) / (2 * (self.counts + 1)))
        k = np.argmax(ucb)
        r = self.bandit.step(k)
        self.estimates[k] += 1. /(self.counts[k] + 1) * (r - self.estimates[k])
        return k

In [ ]:
np.random.seed(1)
coef = 1
UCB_solver = UCB(bandit, coef)
UCB_solver.run(5000)
print('上置信界算法的累计懊悔为：', UCB_solver.regret)
plot_results([UCB_solver], ['UCB'])

In [ ]:
class ThompsonSampling(Solver):
    def __init__(self, bandit):
        super(ThompsonSampling, self).__init__(bandit)
        self._a = np.ones(self.bandit.K)
        self._b = np.ones(self.bandit.K)
        
    def run_one_step(self):
        samples = np.random.beta(self._a, self._b)
        k = np.argmax(samples)
        r = self.bandit.step(k)
        self._a[k] += r
        self._b[k] += (1 - r)
        return k

In [ ]:
np.random.seed(3)
K = 10
bandit = BernoulliBandit(K)
thompson_sampling_solver = ThompsonSampling(bandit)
thompson_sampling_solver.run(1000000)
print('汤普森采样算法的累计懊悔为：', thompson_sampling_solver.regret)
plot_results([thompson_sampling_solver], ['ThompsonSampling'])